In [2]:
import pandas as pd
import os
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [12]:
TwoQs_df = pd.read_excel('/Volumes/LaCie/event_ews/old_culture_data/2013_JHMI_Pascal_all_QualComms.xlsx')
TwoQs_df.head()

,hospital,work_setting,comments,collector
0,Howard County General Hospital,1N/Psych,Injured by another violent or psychotic patien...,download
1,Howard County General Hospital,1N/Psych,We have staff that lie in their reports making...,download
2,Howard County General Hospital,1N/Psych,Assault by another patient.,download
3,Howard County General Hospital,1N/Psych,the patient may be injured by another patient ...,download
4,Howard County General Hospital,1N/Psych,pt. on pt assault. improve staffing;don't admi...,download


In [11]:
TwoQs_df = TwoQs_df[TwoQs_df.hospital == 'JHH']
len(TwoQs_df.work_setting.unique())

4304

In [19]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
TwoQs_df['compound'] = [analyzer.polarity_scores(str(x))['compound'] for x in TwoQs_df['comments']]
TwoQs_df['neg'] = [analyzer.polarity_scores(str(x))['neg'] for x in TwoQs_df['comments']]
TwoQs_df['pos'] = [analyzer.polarity_scores(str(x))['pos'] for x in TwoQs_df['comments']]
TwoQs_df['neu'] = [analyzer.polarity_scores(str(x))['neu'] for x in TwoQs_df['comments']]

In [23]:
TwoQs_vader_metrics = TwoQs_df.groupby('work_setting')[['compound','neg','pos','neu']].agg(['mean','std']).reset_index()
TwoQs_vader_metrics.columns = TwoQs_vader_metrics.columns.map('_'.join).str.strip('_')
TwoQs_vader_metrics.head()

,work_setting,compound_mean,compound_std,neg_mean,neg_std,pos_mean,pos_std,neu_mean,neu_std
0,ACCM CVIL,-0.884200,NaN,0.173000,NaN,0.029000,NaN,0.798000,NaN
1,ACCM Departmental Staff,0.003362,0.561793,0.171524,0.192304,0.107238,0.124057,0.721214,0.198185
2,ACCM JHOC,0.927800,NaN,0.000000,NaN,0.273000,NaN,0.727000,NaN
3,ACCM Weinberg,0.215908,0.519744,0.058500,0.079053,0.142083,0.183099,0.799583,0.203553
4,ACCM Zayed 3,0.016925,0.364530,0.076750,0.107183,0.086500,0.118159,0.836800,0.160069


In [20]:
TwoQs_df['embeddings'] = TwoQs_df['comments'].map(lambda x: model.encode(str(x), convert_to_tensor=False))

In [25]:
def group_sim_mean(x):
    sim = cosine_similarity(list(x))
    np.fill_diagonal(sim,np.nan)
    return sim[~np.isnan(sim)].mean()

def group_sim_std(x):
    sim = cosine_similarity(list(x))
    np.fill_diagonal(sim,np.nan)
    return sim[~np.isnan(sim)].std()

In [28]:
TwoQs_sim_metrics2 = TwoQs_df.groupby('work_setting')['embeddings'].apply(group_sim_mean).reset_index(name='mean_sim')
TwoQs_sim_metrics3 = TwoQs_df.groupby('work_setting')['embeddings'].apply(group_sim_std).reset_index(name='std_sim')

/var/folders/v6/ztvgzd816jd95zsvrnl_zb2c0000gp/T/ipykernel_74881/2835754924.py:4: RuntimeWarning: Mean of empty slice.
  return sim[~np.isnan(sim)].mean()
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/v6/ztvgzd816jd95zsvrnl_zb2c0000gp/T/ipykernel_74881/2835754924.py:4: RuntimeWarning: Mean of empty slice.
  return sim[~np.isnan(sim)].mean()
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/v6/ztvgzd816jd95zsvrnl_zb2c0000gp/T/ipykernel_74881/2835754924.py:4: RuntimeWarning: Mean of empty slice.
  return sim[~np.isnan(sim)].mean()
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value enc

In [30]:
all_TwoQs_metrics = TwoQs_vader_metrics.set_index('work_setting').join(TwoQs_sim_metrics2.set_index('work_setting'))
all_TwoQs_metrics = all_TwoQs_metrics.join(TwoQs_sim_metrics3.set_index('work_setting'))
all_TwoQs_metrics.head()

,compound_mean,compound_std,neg_mean,neg_std,pos_mean,pos_std,neu_mean,neu_std,mean_sim,std_sim
work_setting,,,,,,,,,,
ACCM CVIL,-0.884200,NaN,0.173000,NaN,0.029000,NaN,0.798000,NaN,NaN,NaN
ACCM Departmental Staff,0.003362,0.561793,0.171524,0.192304,0.107238,0.124057,0.721214,0.198185,0.235522,0.174711
ACCM JHOC,0.927800,NaN,0.000000,NaN,0.273000,NaN,0.727000,NaN,NaN,NaN
ACCM Weinberg,0.215908,0.519744,0.058500,0.079053,0.142083,0.183099,0.799583,0.203553,0.194925,0.246191
ACCM Zayed 3,0.016925,0.364530,0.076750,0.107183,0.086500,0.118159,0.836800,0.160069,0.220048,0.211421


In [31]:
all_TwoQs_metrics.to_csv('/Volumes/LaCie/event_ews/all_2Qs_metrics_12-29-2023.csv')